In [31]:
from faker import Faker
import json
import sqlite3
import random
import secrets

In [32]:
# fetch the products id, names, price from the database
def fetch_products():
    conn = sqlite3.connect('database/database.db')
    c = conn.cursor()
    c.execute('SELECT * FROM pricings')
    pricings_table = c.fetchall()
    conn.close()
    return pricings_table

# fetch the customers id, names from the database
def fetch_customers():
    conn = sqlite3.connect('database/database.db')
    c = conn.cursor()
    c.execute('SELECT * FROM customers')
    customers_table = c.fetchall()
    conn.close()
    return customers_table

# fetch products from the database
def fetch_products_table():
    conn = sqlite3.connect('database/database.db')
    c = conn.cursor()
    c.execute('SELECT id, weight FROM products')
    pricings_table = c.fetchall()
    conn.close()
    return pricings_table

# from the product id look up the weight of the product
def get_product_weight(pricings_table, product_id):
    for product in pricings_table:
        if product[0] == product_id:
            return product[1]

# randomly select a customer id from the customers table
def random_customer_id(customers_table):
    return random.choice(customers_table)[0]

# from the customer id get the customer type
def get_customer_type(customers_table, customer_id):
    for customer in customers_table:
        if customer[0] == customer_id:
            return customer[4]
        
# from the customer id get the customer name, if it is a business customer then return the business name
def get_customer_name(customers_table, customer_id):
    for customer in customers_table:
        if customer[0] == customer_id:
            if customer[4] == "Business":
                return customer[3]
            else: # personal
                return f"{customer[1]} {customer[2]}"
        
# from the customer id get the customer address
def get_customer_address(customers_table, customer_id):
    for customer in customers_table:
        if customer[0] == customer_id:
            return customer[5]

# randomly select a product id from the pricings table
def random_product_id(pricings_table):
    return random.choice(pricings_table)[1]

# get the product name from the pricing table
def get_product_name(pricings_table, product_id):
    for product in pricings_table:
        if product[1] == product_id:
            return product[2]

# get the sku_units_ratio from the pricings table
def get_product_sku_units_ratio(pricings_table, product_id):
    for product in pricings_table:
        if product[1] == product_id:
            return product[3]

# get the product price from the pricing table based on the products id and type
def get_product_price(pricings_table, product_id, customer_type):
    for product in pricings_table:
        if product[1] == product_id and customer_type == "Business":
            return product[4]
        else : # personal
            return product[5]

# order id 36 hexes
def order_crypto_id():
    return secrets.token_hex(36)

# order quantity
def order_quantity():
    return random.randint(1, 15)

# order sku
def order_sku():
    sku = [
        'CARTON', # cartons
        'OUTER', # outer
        'BOTTLE', # bottle
        'PACKET', # packet
        'BUNDLE' # bundle
    ]
    
    weight = [
        0.1,
        0.25,
        0.2,
        0.35,
        0.1
    ]
    
    return random.choices(sku, weight,k=1)[0]

# order status
def order_status():
    status = [
        'pending',
        'processing',
        'completed',
        'cancelled'
    ]
    
    weight = [
        0.3,
        0.2,
        0.4,
        0.1
    ]
    
    return random.choices(status, weight, k=1)[0]

# order date
def order_date():
    fake = Faker()
    date_obj = fake.date_this_year()
    return date_obj.strftime('%Y-%m-%d')

In [33]:
def create_order():
    products = fetch_products()
    customers = fetch_customers()
    products_table = fetch_products_table()
    product_id = random_product_id(products)
    customer_id = random_customer_id(customers)
    amount = order_quantity()
    sku_type = order_sku()
    customer_type = get_customer_type(customers, customer_id)
    sku_units_ratio = get_product_sku_units_ratio(products, product_id)
    shipping_address = get_customer_address(customers, customer_id)
    
    if sku_type == 'CARTON' or sku_type == 'BUNDLE':
        price = get_product_price(products, product_id, customer_type)
        total_price = price * amount
        weight = get_product_weight(products_table, product_id) * amount
    else:
        price = get_product_price(products, product_id, customer_type) * sku_units_ratio/100
        total_price = price * amount
        weight = get_product_weight(products_table, product_id) * amount * sku_units_ratio/100
    

    order = {
        'id': order_crypto_id(),
        'date': order_date(),
        'status': order_status(),
        'product_id': product_id,
        'name': get_product_name(products, product_id),
        'quantity': amount,
        'price': price,
        'total_price': total_price,
        'sku': sku_type,
        'weight': weight,
        'customer_id': customer_id,
        'customer_name': get_customer_name(customers, customer_id),
        'customer_type': customer_type,
        'shipping_address': shipping_address,
        'billing_address': shipping_address
    }
    return order

def generate_multiple_orders(n):
    orders = [create_order() for _ in range(n)]
    return json.dumps(orders, indent=4)

In [34]:
# test
print(generate_multiple_orders(1))

[
    {
        "id": "9998997d511981a83b56722ad3593aa0f393833153783c5ae42ab7d850dde9d7a7eb4d16",
        "date": "2024-07-26",
        "status": "completed",
        "product_id": "385093a05a87da912bf0683478fbdbddcbe858d8251fc3d1991acbb02df2a8820f6363a8",
        "name": "Ergonomic Granite Shoes",
        "quantity": 3,
        "price": 9.766079999999999,
        "total_price": 29.298239999999996,
        "sku": "PACKET",
        "weight": 7.164,
        "customer_id": "6f3c3fb344c96bb242361436d4b8715bd6138c9eee57fdba40abfebb6056692868ecff2b",
        "customer_name": "Jeffrey Rivera",
        "customer_type": "Personal",
        "shipping_address": "54905 Sean Isle Apt. 290\nMitchellbury, CO 26751",
        "billing_address": "54905 Sean Isle Apt. 290\nMitchellbury, CO 26751"
    }
]


In [35]:
# insert the orders into the database
conn = sqlite3.connect('database/database.db')
c = conn.cursor()

c.execute(
    '''
    CREATE TABLE IF NOT EXISTS orders (
        id TEXT PRIMARY KEY NOT NULL,
        date DATE NOT NULL,
        status TEXT NOT NULL,
        product_id INTEGER NOT NULL,
        name TEXT NOT NULL,
        quantity INTEGER NOT NULL,
        price REAL NOT NULL,
        total_price REAL NOT NULL,
        sku TEXT NOT NULL,
        weight REAL NOT NULL,
        customer_id INTEGER NOT NULL,
        customer_type TEXT NOT NULL,
        shipping_address TEXT NOT NULL,
        billing_address TEXT NOT NULL,
        FOREIGN KEY (product_id) REFERENCES products (id),
        FOREIGN KEY (customer_id) REFERENCES customers (id)
    )
    '''
)

json_data = generate_multiple_orders(3000)

# save the json data as json
with open('json_data/orders.json', 'w') as f:
    f.write(json_data)
    
for order in json.loads(json_data):
    c.execute(
        '''
        INSERT INTO orders (id, date, status, product_id, name, quantity, price, total_price, sku, weight, customer_id, customer_type, shipping_address, billing_address)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''',
        (
            order['id'],
            order['date'],
            order['status'],
            order['product_id'],
            order['name'],
            order['quantity'],
            order['price'],
            order['total_price'],
            order['sku'],
            order['weight'],
            order['customer_id'],
            order['customer_type'],
            order['shipping_address'],
            order['billing_address']
        )
    )
    
conn.commit()
conn.close()

CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    email TEXT NOT NULL,
    phone_number TEXT,
    address TEXT,
    city TEXT,
    state TEXT,
    postal_code TEXT,
    country TEXT,
    date_of_registration DATE
);

CREATE TABLE products (
    product_id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_name TEXT NOT NULL,
    price DECIMAL(10, 2) NOT NULL,
    category_id INTEGER,
    FOREIGN KEY (category_id) REFERENCES categories(category_id)
);

CREATE TABLE categories (
    category_id INTEGER PRIMARY KEY AUTOINCREMENT,
    category_name TEXT NOT NULL
);

CREATE TABLE order_items (
    order_item_id INTEGER PRIMARY KEY AUTOINCREMENT,
    order_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    quantity INTEGER NOT NULL,
    price DECIMAL(10, 2) NOT NULL,
    FOREIGN KEY (order_id) REFERENCES orders(order_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);

CREATE TABLE payments (
    payment_id INTEGER PRIMARY KEY AUTOINCREMENT,
    order_id INTEGER NOT NULL,
    payment_method TEXT NOT NULL,
    payment_date DATE NOT NULL,
    amount DECIMAL(10, 2) NOT NULL,
    FOREIGN KEY (order_id) REFERENCES orders(order_id)
);

CREATE TABLE shipping (
    shipping_id INTEGER PRIMARY KEY AUTOINCREMENT,
    order_id INTEGER NOT NULL,
    shipping_date DATE,
    delivery_date DATE,
    shipping_cost DECIMAL(10, 2),
    shipping_address TEXT NOT NULL,
    FOREIGN KEY (order_id) REFERENCES orders(order_id)
);

CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id INTEGER NOT NULL,
    order_date DATE NOT NULL,
    order_status TEXT NOT NULL,
    total_amount DECIMAL(10, 2) NOT NULL,
    payment_method TEXT NOT NULL,
    shipping_address TEXT NOT NULL,
    billing_address TEXT NOT NULL,
    shipping_cost DECIMAL(10, 2),
    discount DECIMAL(10, 2),
    tax DECIMAL(10, 2),
    notes TEXT,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);

In [36]:
#conn = sqlite3.connect('database/database.db')
#c = conn.cursor()
#c.execute('DROP TABLE orders')
#conn.commit()